In [1]:
import pandas as pd
import numpy as np


import matplotlib.pyplot as plt

from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

from itertools import combinations

from nets_algo import *

## Функция потерь

Создание функции потерь в torch
https://neptune.ai/blog/pytorch-loss-functions. Для задач бинарной классификации рекомендуют использовать CrossEntropy функцию потерь

In [2]:
loss_fn = nn.BCELoss()

# Загрузка и подговтовка данных

In [3]:
data = pd.read_csv(
    "/home/dranik/KFA/university/proc_2.csv", 
    index_col = 0
)

save_ind = data[data['Y'] == 0].sample(
    sum(data['Y']), random_state = 0
).index.union(data[data['Y'] == 1].index)
data = data.loc[save_ind]

Нужно провести One Hot Encoding

In [4]:
Y = np.array(data[['Y']])
X = data.drop('Y', axis = 1)

X = np.concatenate([
    OneHotEncoder(sparse = False).\
    fit_transform(X.loc[:,X.dtypes == "O"]),
    X.loc[:,X.dtypes != "O"].to_numpy()
], axis = 1)

Y.shape

(13926, 1)

Разбивка на Train/Test

In [5]:
X_train, X_test, y_train, y_test = \
    train_test_split(
        X,Y, random_state = 0, stratify = Y
)

## Создание набора данных и загрузчика данных

In [6]:
train_data = My_data_set(
    torch.tensor(X_train.astype('float32')), 
    torch.tensor(y_train.astype('float32'))
)

In [7]:
train_data_loader =\
torch.utils.data.DataLoader(
    train_data, batch_size=1000
)

# Эксперименты с построителем модели

Модель на которой планируется проводить тестирование

In [8]:
def create_model():
    
    torch.manual_seed(0)
    net = ResultNet(
        [train_data_loader.dataset.X.shape[1], 5]
    )

    optimizer = optim.Adam(
        net.parameters(), 
        weight_decay = 0.5,
        lr = 0.1
    )
    
    return [net, optimizer]

Базовый построитель

In [9]:
def train_basic(
    model, optimizer, loss_fn, 
    train_loader, epochs=20
):
    '''Алгоритм обучения сети'''
    # inputs:
    # model - модель которая подлежит обучению
    # optimizer - оптимизатор, который педполагается использовать
    # loss_fn - функция потерь
    # train_loader - загрузчик обучающих данных
    # epochs - эпохи используемые в нейронной сети
    # lr_decr - степень понижения параметра learning rate
    
    initial_loss = get_loss_value(
        loss_fn, model, train_loader
    )
    
    fun_arr = []
    fun_arr.append(initial_loss)
    
    for epoch in range(epochs):

        model.train()
        for batch in train_loader:
            optimizer.zero_grad()
            inputs, targets = batch
            output = model(inputs)
            loss = loss_fn(output, targets)
            loss.backward()
            optimizer.step()
            
        # для отслеживания процесса
        # обучения буду сохранять текущее
        # значение целевой функии
        fun_arr.append(get_loss_value(
            loss_fn, model, train_loader
        ))

    
    return fun_arr


In [10]:
net, optimizer = create_model()

train_basic(
    net, optimizer, loss_fn, 
    train_data_loader, epochs = 3
)

[28.521753311157227, 21.808807373046875, 10.963159561157227, 7.656264305114746]

Улучшенный построитель с возможностью торможения learning rate

In [12]:
net, optimizer = create_model()

train_basic(
    net, optimizer, loss_fn, 
    train_data_loader, epochs = 3
)

[28.521753311157227, 21.808807373046875, 10.963159561157227, 7.656264305114746]

In [15]:
def train(
    model, optimizer, loss_fn, 
    train_loader, epochs=20, 
    lr_scheduler = None
):
    '''Алгоритм обучения сети'''
    # inputs:
    # model - модель которая подлежит обучению
    # optimizer - оптимизатор, который педполагается использовать
    # loss_fn - функция потерь
    # train_loader - загрузчик обучающих данных
    # epochs - эпохи используемые в нейронной сети
    # lr_scheduler - планировщик learning rate
    
    initial_loss = get_loss_value(
        loss_fn, model, train_loader
    )
    
    fun_arr = []
    fun_arr.append(initial_loss)
    
    for epoch in range(epochs):

        model.train()
        for batch in train_loader:
            optimizer.zero_grad()
            inputs, targets = batch
            output = model(inputs)
            loss = loss_fn(output, targets)
            loss.backward()
            optimizer.step()
        if lr_scheduler:
            lr_scheduler.step()
            
        # для отслеживания процесса
        # обучения буду сохранять текущее
        # значение целевой функии
        fun_arr.append(get_loss_value(
            loss_fn, model, train_loader
        ))

    
    return fun_arr

In [16]:
net, optimizer = create_model()
my_lr_scheduler = optim.lr_scheduler.ExponentialLR(
    optimizer, gamma = 0.1
)

train(
    net, optimizer, loss_fn, 
    train_data_loader, epochs = 3,
    lr_scheduler = my_lr_scheduler
)

[28.521753311157227, 21.808807373046875, 20.100093841552734, 19.85100555419922]

# Массовое построение моделей заданной формы

In [18]:
def model_fit_get_perfomance(
    hidden_layers_ranges, epochs, loss_fn,
    train_loader, X_test, y_test, weight_decay = 0.5, 
    lr = 0.1
):
    '''Метод обеспечивает построение моделей заданных форм
    и "снимает" инофрмацию о их свойсвах'''
    # inputs:
    # hidden_layers_ranges - инофрмация о внутренних
    #                        слоях модели как list
    #                        который содержит также списки
    #                        типа [<нейноны 1 слоя>, <нейноны 2 слоя>, ...]
    # lr -                   learning rate с которого начинается
    #                        оптимизация
    # epochs -               сколько эпох проходит каждая из 
    #                        форм модели
    # test_X, test_y -       данные на которых проводиться валидация
    #                        модели
    # train_loader -         загрузчик трерировочных данных
    # weight_decay -         параметр регуляризации
    # output:
    # [learning_info, auc_info, nets]
    # learning_info -        pandas.DataFrame по столбцам значения целевой
    #                        функции для эпохи в строке
    # auc_info -             pandas.Series AUC на тестовых данных для
    #                        каждой формы модели
    # nets -                 список с полученными моделями - с перспективой
    #                        дообучения
    
    
    # выходные массивы
    learning_info = pd.DataFrame()
    auc_info = pd.Series(dtype = 'float64')
    nets = []
    
    
    for layers_info in hidden_layers_ranges:
        # применение текущих настроек =================
        net = ResultNet(
            [train_loader.dataset.X.shape[1]] + 
            layers_info
        )
        optimizer = optim.Adam(
            net.parameters(), 
            weight_decay = weight_decay, 
            lr = lr
        )
        # применение текущих настроек =================
        # обучение ====================================
        #print(net.layers[0].bias)
        lc = train(
            net, optimizer, loss_fn, 
            train_loader, epochs = epochs
        )
        # обучение ====================================
        # снятие метрик ===============================
        probs_hat = net(
            torch.tensor(X_test.astype('float32'))
        ).detach().numpy()
    
        auc = roc_auc_score(y_test, probs_hat)
        # снятие метрик ===============================
        # сохранение информации =======================
        
        identyfyer = str(layers_info)
        learning_info.loc[:, identyfyer] = lc
        auc_info[identyfyer] = auc
        nets.append(net)
        
        # сохранение информации =======================
        print(identyfyer + " AUC = " + str(auc))
        
    return [learning_info, auc_info, nets]

In [19]:
model_fit_get_perfomance(
    [[4], [3,3]], 5, loss_fn,
    train_data_loader, X_test, y_test,
)

[4] AUC = 0.4916189966549887
[3, 3] AUC = 0.58868667646955


[         [4]     [3, 3]
 0  40.220894  26.253555
 1  10.559843   6.866017
 2   7.039437   3.470518
 3   5.462039   0.694448
 4   3.094617   0.693317
 5   3.012087   0.695041,
 [4]       0.491619
 [3, 3]    0.588687
 dtype: float64,
 [ResultNet(
    (layers): Sequential(
      (0): Linear(in_features=108, out_features=4, bias=True)
      (1): Linear(in_features=4, out_features=1, bias=True)
    )
  ),
  ResultNet(
    (layers): Sequential(
      (0): Linear(in_features=108, out_features=3, bias=True)
      (1): Linear(in_features=3, out_features=3, bias=True)
      (2): Linear(in_features=3, out_features=1, bias=True)
    )
  )]]